In [1]:
from IPython.core.display import HTML

# See https://github.com/bstriner/keras-tqdm/issues/21#issuecomment-443019223
display(HTML("""
    <style>
        .jp-OutputArea-child:has(.jp-OutputArea-prompt:empty) {
              padding: 0 !important;
        }
    </style>
"""))

In [2]:
import pickle
import pandas as pd
import os

import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm, trange
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint
import matplotlib.pyplot as plt
import torch

from sklearn.preprocessing import MinMaxScaler
import math
from torch.profiler import profile, record_function, ProfilerActivity

from operator import itemgetter
import operator
from random import randint
from rnn_classes import Dog, DogInput, Race, Races, GRUNet
from raceDB import build_dataset, build_pred_dataset
import importlib
import datetime


<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [4]:
torch.set_default_tensor_type(torch.FloatTensor)

In [5]:
torch.__version__

'1.12.1'

In [6]:
def model_saver(model, optimizer, epoch, loss, hidden_state_dict, model_name = None):
    
    pathtofolder = "C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model"
    # model_name = wandb.run.name
    if not model_name:
        model_name = "test NZ GRU saver"
    isExist = os.path.exists(
        f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/"
    )
    if isExist:
        torch.save(
            {
                "epoch": epoch,
                "model_state_dict": model.state_dict(),
                "optim": optimizer.state_dict(),
                "loss": loss,
                "db":hidden_state_dict,
            },
            f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/{model_name}_{epoch}.pt",
        )
    else:
        print("created path")
        os.makedirs(
            f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/"
        )
        torch.save(
            {
                "epoch": epoch,
                "model_state_dict": model.state_dict(),
                "optim": optimizer.state_dict(),
                "loss": loss,
                "db":hidden_state_dict,
            },
            f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/{model_name}_{epoch}.pt",
        )

In [7]:
def validation_CLE(x,y):
    loss_t = -torch.log(torch.exp(x)/torch.sum(torch.exp(x), dim=-1, keepdim=True))*y
    return loss_t

In [8]:
#Testing
def validate_model(model:GRUNet,raceDB,criterion, batch_size, example_ct, epoch_loss, batch_ct):
    torch.autograd.set_detect_anomaly(True)
    sft_max = nn.Softmax(dim=-1)
    batch_size=10
    len_test = len(raceDB.test_race_ids)-batch_size
    list_t = [] 
    last = 0
    loss_val = 0 
    correct = 0
    total = 0
    model.eval()
    actuals = []
    preds = []
    grades = []
    tracks = []
    pred_confs = []
    bfsps = []
    start_prices = []
    loss_l = []
    loss_t = []
    margins_l = []
    preds_l = []
    pred_sftmax = []
    raw_margins = []
    raw_places = []
    margins_prob = []

    prices_list = []
    raw_prices = []

    price_dict = {}
    price_dict['prices'] = []
    price_dict['imp_prob'] = []
    price_dict['pred_prob'] = []
    price_dict['pred_price'] = []
    price_dict['margin'] = []
    price_dict['onehot_win'] = []
    price_dict['raceID'] = []
    price_dict['dogID'] = []
    race_ids = []
    with torch.no_grad():
        for i in trange(0,len_test,batch_size, leave=False):
            races_idx = range(last,last+batch_size)
            last = i+batch_size
            race = raceDB.get_test_input(races_idx)
            #tracks.extend([r.track for r in race])

            X = race
            y = torch.stack([x.classes for x in race])
            output = model(X)
            race_ids.extend([x.raceid for x in race])
            #print(y)

            _, actual = torch.max(y.data, 1)
            onehot_win = F.one_hot(actual, num_classes=8)
            conf, predicted = torch.max(output.data, 1)
            correct += (predicted == actual).sum().item()

            softmax_preds = sft_max(output)

            
            total += batch_size
            actuals.extend(actual.tolist())
            preds.extend(predicted.tolist())
            pred_confs.extend(conf.tolist())
            tracks.extend([r.track_name for r in race])
            grades.extend([r.grade for r in race])
            for i,dog_idx in enumerate(actual.tolist()):
                bfsps.append(race[i].dogs[dog_idx].bfsp)
                #start_prices.append(race[i].dogs[dog_idx].sp)

            
            loss = criterion(output, y).detach()
            loss_tensor = validation_CLE(output,y)
            loss_t.append(loss_tensor.tolist())

            loss_l.append(loss.tolist())
            preds_l.append(output.tolist())
            pred_sftmax.append(softmax_preds.tolist())
            margins_l.append(y.tolist())
            margins_prob.append(y.tolist())
            raw_margins.append([x.raw_margins for x in race])
            raw_places.append([x.raw_places for x in race])
            loss_val += loss

            price_dict['prices'].extend([x.prices for x in race])
            price_dict['imp_prob'].extend([x.implied_prob for x in race])
            price_dict['pred_prob'].extend(softmax_preds.tolist())
            #print([(1/(x+(-7**10))).tolist() for x in torch.exp(output)])
            price_dict['pred_price'].extend([(1/(x)).tolist() for x in softmax_preds])
            price_dict['margin'].extend([x.raw_margins for x in race])
            price_dict['onehot_win'].extend(onehot_win.tolist())
            price_dict['raceID'].extend([[x.raceid]*8 for x in race])
            price_dict['dogID'].extend([x.list_dog_ids() for x in race])

            

        loss_list = []

        #print("start loss calc")
        for i,l in enumerate(loss_l):
            for j in range(0,7):
                loss_list.append([preds_l[i][j],margins_l[i][j],loss_t[i][j],l[j],pred_sftmax[i][j],margins_prob[i][j], raw_margins[i][j], raw_places[i][j]])

    loss_df = pd.DataFrame(loss_list, columns=['Preds','Margins','Indiviual Losses','Losses','softmaxPreds','Softmax Margins','Raw margins', 'Raw places'])
    loss_table = wandb.Table(dataframe=loss_df)


    logdf = pd.DataFrame(data = {"actuals":actuals, "preds":preds,"conf":pred_confs, "grade":grades, "track":tracks, "bfsps":bfsps})#, "sp":start_prices })
    
    logdf['correct'] = logdf.apply(lambda x: 1 if x['actuals']==x['preds'] else 0, axis=1)
    logdf['profit'] = logdf.apply(lambda x: 0 if x['bfsps']<1 else x['bfsps']-1  if x['correct'] else -1, axis=1)
    logdf.to_csv('logDFtest.csv')

    table = wandb.Table(dataframe=logdf)

    logdf['count'] = 1
    logdf['eligible_ct'] = logdf.apply(lambda x: 1 if x['bfsps'] > 1 else 0, axis = 1)

    track_df = logdf.groupby('track', as_index=False).sum().reset_index()
    track_table = wandb.Table(dataframe=track_df)
    
    prices_df = pd.DataFrame(price_dict)


    prices_df['sum_price'] = prices_df.apply(lambda x: sum(x['prices']), axis = 1)
    prices_df = prices_df[prices_df['sum_price']>0]

    prices_flat = [item for sublist in prices_df['prices'].tolist() for item in sublist]
    pred_prices = [item for sublist in prices_df['pred_price'].tolist() for item in sublist]
    onehot_win  = [item for sublist in prices_df['onehot_win'].tolist() for item in sublist]
    flat_margins = [item for sublist in prices_df['margin'].tolist() for item in sublist]
    flat_dogs = [item for sublist in prices_df['dogID'].tolist() for item in sublist]
    flat_races = [item for sublist in prices_df['raceID'].tolist() for item in sublist]
    all_price_df = pd.DataFrame(data={'flat_dogs':flat_dogs,'flat_races':flat_races,'prices':prices_flat, 'pred_price':pred_prices, 'onehot_win':onehot_win,'split_margin':flat_margins})
    all_price_df = all_price_df[all_price_df['prices']>1]
    all_price_df['imp_prob'] =  all_price_df.apply(lambda x: 1/x['prices'], axis = 1)
    all_price_df['pred_prob'] =  all_price_df.apply(lambda x: 1/x['pred_price'], axis = 1)
    all_price_df['bet amount'] = all_price_df.apply(lambda x: (x['pred_prob']-x['imp_prob'])*100 if (x['pred_prob']>x['imp_prob'])&(1>x['imp_prob']>0) else 0, axis = 1)
    all_price_df['profit'] = all_price_df.apply(lambda x: x['bet amount']*(x['prices']-1)*0.95 if x['onehot_win'] else -1*x['bet amount'], axis = 1)
    all_price_df['flat_profit'] = all_price_df.apply(lambda x: 1*(x['prices']-1)*0.95 if (x['onehot_win'] and x['bet amount']) else -1, axis = 1)
    all_price_df['colour'] = all_price_df.apply(lambda x: "profitz" if x['profit']>0 else ("loss" if x['profit']<0 else ("no bet - win" if x['onehot_win'] else "no bet")), axis=1)

    all_price_table = wandb.Table(dataframe=all_price_df)


    price_table = wandb.Table(dataframe=prices_df)

    try:
        wandb.log({"table_key": table,"loss_table": loss_table,"track_df": track_table,"price_table":price_table, 'allprice_df':all_price_df })
    except Exception as e:
        print(e)

    #print(f"accuray: {correct/total}")
    wandb.log({"accuracy": correct/total, "loss_val": torch.mean(loss_val)/len_test, "correct": correct, 'profit': logdf[logdf['actuals']==logdf['preds']]['profit'].sum(), 'multibet profit':all_price_df['profit'].sum(), 'multibet outlay':all_price_df['bet amount'].sum() })

    return correct/total

In [9]:
#Testing
def validate_modelv2(model:GRUNet,raceDB:Races):
    torch.autograd.set_detect_anomaly(True)
    sft_max = nn.Softmax(dim=-1)
    batch_size=10
    len_test = len(raceDB.raceIDs)-batch_size
    list_t = [] 
    last = 0
    loss_val = 0 
    correct = 0
    total = 0
    model.eval()
    actuals = []
    preds = []
    grades = []
    tracks = []
    pred_confs = []
    bfsps = []
    start_prices = []
    loss_l = []
    loss_t = []
    margins_l = []
    preds_l = []
    pred_sftmax = []
    raw_margins = []
    raw_places = []
    margins_prob = []

    prices_list = []
    raw_prices = []

    price_dict = {}
    price_dict['prices'] = []
    price_dict['imp_prob'] = []
    price_dict['pred_prob'] = []
    price_dict['pred_price'] = []
    price_dict['margin'] = []
    price_dict['onehot_win'] = []
    with torch.no_grad():
        for i in trange(0,len_test,batch_size, leave=False):
            races_idx = range(last,last+batch_size)
            last = i
            race = raceDB.get_race_input(races_idx)
            #tracks.extend([r.track for r in race])
            X = race
            y = torch.stack([x.classes for x in race])
            output = model(X)
            #print(y)

            _, actual = torch.max(y.data, 1)
            onehot_win = F.one_hot(actual, num_classes=8)
            conf, predicted = torch.max(output.data, 1)
            correct += (predicted == actual).sum().item()

            softmax_preds = sft_max(output)

            
            total += batch_size
            actuals.extend(actual.tolist())
            preds.extend(predicted.tolist())
            pred_confs.extend(conf.tolist())
            tracks.extend([r.track_name for r in race])
            grades.extend([r.grade for r in race])
            for i,dog_idx in enumerate(actual.tolist()):
                bfsps.append(race[i].dogs[dog_idx].bfsp)
                #start_prices.append(race[i].dogs[dog_idx].sp)


            price_dict['prices'].extend([x.prices for x in race])
            price_dict['imp_prob'].extend([x.implied_prob for x in race])
            price_dict['pred_prob'].extend(softmax_preds.tolist())
            #print([(1/(x+(-7**10))).tolist() for x in torch.exp(output)])
            price_dict['pred_price'].extend([(1/(x)).tolist() for x in softmax_preds])
            price_dict['margin'].extend([x.raw_margins for x in race])
            price_dict['onehot_win'].extend(onehot_win.tolist())

            


    logdf = pd.DataFrame(data = {"actuals":actuals, "preds":preds,"conf":pred_confs, "grade":grades, "track":tracks, "bfsps":bfsps})#, "sp":start_prices })
    
    logdf['correct'] = logdf.apply(lambda x: 1 if x['actuals']==x['preds'] else 0, axis=1)
    logdf['profit'] = logdf.apply(lambda x: 0 if x['bfsps']<1 else x['bfsps']-1  if x['correct'] else -1, axis=1)
    logdf.to_csv('logDFtest.csv')

    table = wandb.Table(dataframe=logdf)

    logdf['count'] = 1
    logdf['eligible_ct'] = logdf.apply(lambda x: 1 if x['bfsps'] > 1 else 0, axis = 1)

    track_df = logdf.groupby('track', as_index=False).sum().reset_index()
    
    prices_df = pd.DataFrame(price_dict)


    prices_df['sum_price'] = prices_df.apply(lambda x: sum(x['prices']), axis = 1)
    prices_df = prices_df[prices_df['sum_price']>0]

    prices_flat = [item for sublist in prices_df['prices'].tolist() for item in sublist]
    pred_prices = [item for sublist in prices_df['pred_price'].tolist() for item in sublist]
    onehot_win  = [item for sublist in prices_df['onehot_win'].tolist() for item in sublist]
    flat_margins = [item for sublist in prices_df['margin'].tolist() for item in sublist]
    all_price_df = pd.DataFrame(data={'prices':prices_flat, 'pred_price':pred_prices, 'onehot_win':onehot_win,'split_margin':flat_margins})
    all_price_df = all_price_df[all_price_df['prices']>1]
    all_price_df['imp_prob'] =  all_price_df.apply(lambda x: 1/x['prices'], axis = 1)
    all_price_df['pred_prob'] =  all_price_df.apply(lambda x: 1/x['pred_price'], axis = 1)
    all_price_df['bet amount'] = all_price_df.apply(lambda x: (x['pred_prob']-x['imp_prob'])*100 if (x['pred_prob']>x['imp_prob'])&(1>x['imp_prob']>0) else 0, axis = 1)
    all_price_df['profit'] = all_price_df.apply(lambda x: x['bet amount']*(x['prices']-1)*0.95 if x['onehot_win'] else -1*x['bet amount'], axis = 1)
    all_price_df['flat_profit'] = all_price_df.apply(lambda x: 1*(x['prices']-1)*0.95 if (x['onehot_win'] and x['bet amount']) else -1, axis = 1)
    all_price_df['colour'] = all_price_df.apply(lambda x: "profitz" if x['profit']>0 else ("loss" if x['profit']<0 else ("no bet - win" if x['onehot_win'] else "no bet")), axis=1)


    #print(f"accuray: {correct/total}")
    stats = {"accuracy": correct/total,  "correct": correct, 'profit': logdf[logdf['actuals']==logdf['preds']]['profit'].sum(), 'multibet profit':all_price_df['profit'].sum(), 'multibet outlay':all_price_df['bet amount'].sum() }
    print(stats)
    return all_price_df

In [10]:
def train_log(loss, example_ct, epoch):
    # Where the magic happens
    wandb.log({"epoch": epoch, "loss": loss}, step=example_ct)

In [27]:
def train(model, raceDB:Races, criterion, optimizer,scheduler, config=None):
    torch.autograd.set_detect_anomaly(True)

    last = 0
    batch_size = 25
    len_train = len(raceDB.train_race_ids)-batch_size
    example_ct = 0  # number of examples seen
    batch_ct = 0
    m = nn.LogSoftmax(dim=1)
    epoch_loss=0
    batch_before_backwards = 10
    saved_batches = []
    #losses = torch.tesnor()
    for epoch in trange(2000):
        model.train()
        batch_ct = 0
        setup_loss = 1
        batch_size = randint(500,2000)
        len_train = len(raceDB.train_race_ids)-batch_size
        for i in trange(0,len_train,batch_size, leave=False):
            last = i
            #print(f"{i=}\n{batch_ct=}, {setup_loss=}, {batch_ct+1%10==0=}")
            if ((batch_ct+1)%batch_before_backwards==0):
                if setup_loss:
                    print("hit here")
                    continue
                else:
                    epoch_loss = torch.mean(epoch_loss)
                    optimizer.zero_grad()
                    epoch_loss.backward()
                    optimizer.step()
                    model.zero_grad()
                    raceDB.detach_all_hidden_states()
                    setup_loss = 1
                    wandb.log({"batch_loss": epoch_loss.item(), "batch_before_backwards": batch_before_backwards}, step = example_ct)
                    batch_before_backwards = randint(5,20)
                    model(saved_batches)
                    saved_batches = []

            batch_ct += 1

            races_idx = range(last,last+batch_size)
            race = raceDB.get_train_input(races_idx)
            X = race
            saved_batches.extend(X)
            y = torch.stack([x.classes for x in race])
            w = torch.stack([x.win_weight for x in race])
            _, actual = torch.max(y.data, 1)
            output = model(X)

            
            example_ct +=  batch_size

            loss = criterion(output, y)*w
            if setup_loss:
                epoch_loss = loss
                setup_loss=0

            if ((batch_ct + 1) % 25) == 0:                    
                    train_log(torch.mean(loss), example_ct, epoch)

            #epoch_loss = torch.stack([epoch_loss, loss])
            epoch_loss = epoch_loss + loss
            

        #print("finished epoch")
        setup_loss = 1

        wandb.log({"epoch_loss": torch.mean(epoch_loss)}, step = example_ct)
        acc = validate_model(model,raceDB,criterion, 8, example_ct, epoch_loss, batch_ct)
        epoch_loss = 0  
        #print(acc)
        scheduler.step(acc)
    #print(losses)
    return model



In [12]:
os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")
dog_stats_file = pd.read_pickle('new gru input 2023-01.npy')

In [13]:
dog_stats_df = dog_stats_file
dog_stats_df['track_name'] = dog_stats_df[dog_stats_df['track_name'].notna()]['track_name']
dog_stats_df['track_name'] = dog_stats_df['track_name'].dropna()

In [14]:

os.getcwd()
os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")
#dog_stats_file = open( 'new gru input 2023-01.npy', 'rb')
hidden_size = 64
raceDB = build_dataset('new gru input 2023-01.npy', hidden_size ,track_filer=".")

(1105508, 15)
(1105508, 15)
num_features_per_dog=16


  0%|          | 0/39367 [00:00<?, ?it/s]

  0%|          | 0/155094 [00:00<?, ?it/s]

number of races = 155094, number of unique dogs = 39367


In [15]:
date = datetime.datetime.strptime("2022-12-15", "%Y-%m-%d")
raceDB.create_test_split_date(date)

Train examples 151326, Test examples 3768


In [16]:


# dog_stats_file = open( r'C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA\prediction validation 2023-01.npy', 'rb')
# hidden_size = 64
# predDB = build_dataset(dog_stats_file, hidden_size)

In [17]:
# #os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")
# dog_stats_file = open( r'C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA\forms\prediction_inputs_gru.npy', 'rb')
# hidden_size = 64
# predDB = build_pred_dataset(dog_stats_file, hidden_size)

In [18]:
# raceDB.create_hidden_states_dict()
# predDB.fill_hidden_states_from_dict(hidden_dict=raceDB.hidden_states_dict)
# # len(raceDB.hidden_states_dict)
# # predDB.to_cuda()

In [19]:
#Testing
def predict_model(model,predDB):
    torch.autograd.set_detect_anomaly(True)
    list_t = [] 
    last = 0
    loss_val = 0 
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():

        races_idx = range(0,len(predDB.raceIDs)-1)
        race = predDB.get_race_input(races_idx)
        X = race
        # for i,r in enumerate(race):
        #     print(r.raceid, r.track_name)
        #     #print(i,r.lstm_input())

        output = model(X)
        
        print(output)

        _, predicted = torch.max(output.data, 1)

        for i,r in enumerate(race):
            print(r.raceid, r.track_name, r.dogs[predicted[i].item()])

        print(predicted)



In [20]:
def closure(optimizer, criterion, outs, classes):
    optimizer.zero_grad()
    loss = nn.functional.mse_loss(outs, classes)
    loss.backward()
    return loss

def model_pipeline(my_dataset=raceDB,config=None,prev_model=None, sweep=True, model_state_dict=None):
    if my_dataset:
      dataset = my_dataset    
    else:
      dataset = raceDB
    # tell wandb to get started
    with wandb.init(project="GRU - FastTrack - NZ Testing", config=config):
      #  access all HPs through wandb.config, so logging matches execution!
      wandb.define_metric("loss", summary="min")
      wandb.define_metric("test_accuracy", summary="max")
      wandb.define_metric("bfprofit", summary="max")
      config = wandb.config
      pprint.pprint(config)
      pprint.pprint(config.epochs)
      print(config)

      model = GRUNet(203,config['hidden_size'])
      if model_state_dict:
        model.load_state_dict(model_state_dict)
      #criterion = nn.KLDivLoss(reduction='none', log_target=True)
      criterion = nn.CrossEntropyLoss(reduction='none', label_smoothing=0.001)
      optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])

      scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max',threshold=0.001, patience=50, verbose=True, factor=0.5)
      model = model.to(device)
      # optimizer = optimizer.to(device)
      print(model)

      # and use them to train the model
      try:
        train(model, dataset, criterion, optimizer, scheduler, config)
      except KeyboardInterrupt:
        print("finished Early")
      dataset.create_hidden_states_dict()
      # if sweep:
    #   raceDB.reset_all_lstm_states
    


    # and test its final performance
    #test(model, test_loader)

    return (model,dataset, optimizer)

In [21]:
len(raceDB.raceIDs)
wandb_config_static = {'hidden_size':hidden_size,'batch_size': 500, 'dropout': 0.3, 'epochs': 1000, 'f1_layer_size': 256, 'f2_layer_size': 64 , 'learning_rate': 0.0001, 'loss': 'L1', 'l1_beta':0.1,  'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1}

In [22]:
# raceDB.detach_all_hidden_states()
# raceDB.create_test_split()
# raceDB.reset_all_lstm_states()

In [23]:
# save_path = r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\pytorch\New Model\savedmodel\Good profit NZ before update.pt\Good profit NZ before update.pt_450.pt"
# saved_dict = torch.load(save_path)
# # model_state_dict = saved_dict['model_state_dict']
# raceDB.fill_hidden_states_from_dict(saved_dict['db'])
# raceDB.to_cuda()
# # predDB.fill_hidden_states_from_dict(hidden_dict=raceDB.hidden_states_dict)
# # predDB.to_cuda()

In [ ]:
(model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static)

In [ ]:
stop
model = GRUNet(203,64).to(device)
model.load_state_dict(model_state_dict)

<All keys matched successfully>

In [ ]:
all_price_df = validate_modelv2(model, predDB)

  0%|          | 0/27 [00:00<?, ?it/s]

{'accuracy': 0.22962962962962963, 'correct': 62, 'profit': 155.8730624273856, 'multibet profit': 610.4800603151755, 'multibet outlay': 5630.432833115364}


In [ ]:
all_price_df.colour.value_counts()

loss            810
no bet          380
no bet - win    135
profitz          58
Name: colour, dtype: int64

In [ ]:
pause
(model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static, prev_model=model)

NameError: name 'pause' is not defined

In [ ]:
# 
# 
# all_price_df[all_price_df['onehot_win']==1].sort_values('prices', ascending=False).shape
all_price_df.sort_values('prices', ascending=False).shape

(1383, 10)

: 

In [ ]:

dataset.create_hidden_states_dict()

In [29]:
model_saver(model, optimizer, 450, 0.1, dataset.hidden_states_dict, model_name="All races long run 1k epochs")

created path


In [ ]:
sweep_config = {"method": "random"}

metric = {"name": "profit", "goal": "maximize"}

sweep_config["metric"] = metric


parameters_dict = {
    "optimizer": {"value": "adamW"},
    "f1_layer_size": {"values": [256]},
    "f2_layer_size": {"values": [64]},
    "dropout": {"values": [0.3]},
    "len_data": {"value": 70000},
    "hidden_size": {"value":16}
}

sweep_config["parameters"] = parameters_dict

parameters_dict.update(
    {
        "epochs": {"values": [250]},
        "validation_split": {"value": 0.1},
        "loss": {
            "values": [ "L1"],
            # "values": ["Huber", "MSE", "L1", "BCE", "Custom", "KL"]
            # 'value': 'l1_custom'
        },
        "num_layers": {"values": [2]},
    }
)

parameters_dict.update(
    {
        "learning_rate": {
            # a flat distribution between 0 and 0.1
            "distribution": "uniform",
            "min": 0.00011,
            "max": 0.001,
        },
        "l1_beta": {"value": 0.1
        },
        "batch_size": {
            'values': [500,1000,5000]
            # "values": [32, 64, 128, 360, 720]
            # 'values':[4,8,16,32,64,128,360]
        },
    }
)

import pprint

pprint.pprint(sweep_config)


sweep_config


In [ ]:
sweep_id = wandb.sweep(sweep_config, project="LSTM_sweeps")
CUDA_LAUNCH_BLOCKING=1
wandb.agent(sweep_id, function=model_pipeline, count=100)